# 📚 The Great Gatsby - Vector Database Project

This notebook demonstrates how to convert F. Scott Fitzgerald's "The Great Gatsby" into a searchable vector database using ChromaDB and perform semantic search queries.

## What You'll Learn:
- Loading and preprocessing text data
- Creating embeddings with SentenceTransformers
- Storing vectors in ChromaDB
- Performing semantic search queries

## 1. Install and Import Required Libraries

In [ ]:
# Install required packages (uncomment if needed)
# !pip install chromadb sentence-transformers

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
from pathlib import Path
import requests
import re
from pprint import pprint

print("✅ All libraries imported successfully!")

## 2. Download The Great Gatsby Text

We'll download the text from Project Gutenberg (public domain).

In [ ]:
# Download The Great Gatsby from Project Gutenberg
url = "https://www.gutenberg.org/files/64317/64317-0.txt"
data_path = Path("data/great_gatsby.txt")

# Create data directory if it doesn't exist
data_path.parent.mkdir(exist_ok=True)

# Download the file if it doesn't exist
if not data_path.exists():
    print("📥 Downloading The Great Gatsby...")
    response = requests.get(url)
    response.raise_for_status()
    
    with open(data_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print("✅ Downloaded successfully!")
else:
    print("✅ File already exists!")

# Read the file
with open(data_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

print(f"📊 Total characters: {len(raw_text)}")
print(f"📊 First 500 characters:\n{raw_text[:500]}")

## 3. Clean and Preprocess the Text

Remove Project Gutenberg headers/footers and clean the text.

In [ ]:
# Clean the text - remove Project Gutenberg header and footer
def clean_text(text):
    # Find the start of the actual book (after the Gutenberg header)
    start_markers = ["CHAPTER I", "Chapter I", "CHAPTER 1"]
    start_idx = 0
    for marker in start_markers:
        idx = text.find(marker)
        if idx != -1:
            start_idx = idx
            break
    
    # Find the end (before Gutenberg footer)
    end_markers = ["End of Project Gutenberg", "*** END OF THE PROJECT GUTENBERG"]
    end_idx = len(text)
    for marker in end_markers:
        idx = text.find(marker)
        if idx != -1:
            end_idx = idx
            break
    
    # Extract the main text
    clean = text[start_idx:end_idx]
    
    # Clean up extra whitespace
    clean = re.sub(r'\n{3,}', '\n\n', clean)
    clean = re.sub(r' {2,}', ' ', clean)
    
    return clean.strip()

cleaned_text = clean_text(raw_text)
print(f"📊 Cleaned text length: {len(cleaned_text)} characters")
print(f"📊 First 500 characters:\n{cleaned_text[:500]}")

## 4. Split Text into Chunks

We'll split the text into paragraphs for better semantic search results.

In [ ]:
# Split into paragraphs (chunks)
def split_into_chunks(text, min_length=100):
    """Split text into paragraphs, filtering out very short ones."""
    # Split by double newlines (paragraphs)
    paragraphs = text.split('\n\n')
    
    # Filter out very short paragraphs
    chunks = [p.strip() for p in paragraphs if len(p.strip()) >= min_length]
    
    return chunks

chunks = split_into_chunks(cleaned_text)
print(f"📊 Number of chunks: {len(chunks)}")
print(f"📊 Average chunk length: {sum(len(c) for c in chunks) // len(chunks)} characters")
print(f"\n📝 Sample chunk (first one):\n{chunks[0][:300]}...")
print(f"\n📝 Sample chunk (middle one):\n{chunks[len(chunks)//2][:300]}...")

## 5. Initialize ChromaDB Client

Create a persistent client to store our vector database locally.

In [ ]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./gatsby_vector_db")
print("✅ ChromaDB client initialized!")

## 6. Create Sentence Transformer Embedding Function

Initialize the embedding model using SentenceTransformers.

In [ ]:
# Create embedding function using SentenceTransformer
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

print("✅ Sentence Transformer embedding function created!")
print("📊 This model creates 384-dimensional embeddings")

## 7. Create Vector Embeddings

Generate embeddings for all text chunks. This may take a minute...

In [ ]:
# Generate embeddings for all chunks
print(f"🔄 Generating embeddings for {len(chunks)} chunks...")
vectors = sentence_transformer_ef(chunks)

print(f"✅ Generated {len(vectors)} embeddings!")
print(f"📊 Each embedding has {len(vectors[0])} dimensions")

## 8. Store Embeddings in ChromaDB Collection

Create a collection and add all chunks with their embeddings.

In [ ]:
# Create unique IDs for each chunk
ids = [f"chunk_{i}" for i in range(len(chunks))]

# Uncomment to delete existing collection if needed
# chroma_client.delete_collection(name="great_gatsby")

# Create or get collection
collection = chroma_client.get_or_create_collection(name="great_gatsby")

print(f"📦 Adding {len(chunks)} chunks to ChromaDB...")

# Add documents to collection
collection.add(
    documents=chunks,
    ids=ids,
    embeddings=vectors,
)

print(f"✅ Successfully added documents to collection!")
print(f"📊 Collection count: {collection.count()}")

## 9. Query the Vector Database

Now let's search for relevant passages using semantic search!

In [ ]:
# Query 1: Find passages about Gatsby's parties
query = "Gatsby's extravagant parties with music and dancing"
query_embedding = sentence_transformer_ef([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=5, # how many results to return
)

print("🔍 Query:", query)
print("\n" + "="*80)
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n📄 Result {i+1} (Similarity: {1-distance:.4f})")
    print("-" * 80)
    print(doc[:300] + "..." if len(doc) > 300 else doc)
    print()

In [ ]:
# Query 2: Find passages about the green light
query = "the green light at the end of the dock"
query_embedding = sentence_transformer_ef([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
)

print("🔍 Query:", query)
print("\n" + "="*80)
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n📄 Result {i+1} (Similarity: {1-distance:.4f})")
    print("-" * 80)
    print(doc[:400] + "..." if len(doc) > 400 else doc)
    print()

In [ ]:
# Query 3: Find passages about Daisy and Tom
query = "Daisy and Tom Buchanan relationship"
query_embedding = sentence_transformer_ef([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
)

print("🔍 Query:", query)
print("\n" + "="*80)
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n📄 Result {i+1} (Similarity: {1-distance:.4f})")
    print("-" * 80)
    print(doc[:400] + "..." if len(doc) > 400 else doc)
    print()

## 10. Try Your Own Queries!

Modify the query below to search for any topic in The Great Gatsby.

In [ ]:
# 🎯 Try your own query here!
# Suggestions: "Nick Carraway narrator", "valley of ashes", "Jordan Baker golf", 
#              "Gatsby's past", "car accident", "eyes of Doctor T.J. Eckleburg"

query = "eyes of Doctor T.J. Eckleburg watching over everything"
query_embedding = sentence_transformer_ef([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=5,
)

print("🔍 Query:", query)
print(f"\n📊 Found {len(results['documents'][0])} results\n")
print("="*80)
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n📄 Result {i+1} (Similarity: {1-distance:.4f})")
    print("-" * 80)
    print(doc[:350] + "..." if len(doc) > 350 else doc)
    print()

## 11. Alternative Query Method

You can also use `query_texts` instead of creating embeddings manually. ChromaDB will create embeddings automatically.

In [ ]:
# Alternative method - let ChromaDB handle the embedding
results = collection.query(
    query_texts=["Gatsby's mysterious wealth and background"],
    n_results=3,
)

pprint(results)

## 12. Database Statistics

Let's check the statistics of our vector database.

In [ ]:
print("📊 Vector Database Statistics")
print("="*80)
print(f"Collection Name: {collection.name}")
print(f"Total Documents: {collection.count()}")
print(f"Embedding Dimensions: 384")
print(f"Embedding Model: all-MiniLM-L6-v2")
print(f"Storage Path: ./gatsby_vector_db")
print("\n✅ The Great Gatsby is now fully searchable as a vector database!")